In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1609872/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_1609872/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_3_1,0.802722,0.106688,-1.158105,0.669494,0.489948,1.790813,1.591415,1.346343e+00,4.400123e+14,0.699963,0.323617,0.729762,35.426912,56.147802,"Hidden Size=[4], regularizer=0.02, learning_ra..."
1,model_1_3_2,0.805376,0.103512,-1.162003,0.672212,0.483356,1.797181,1.594289,1.335271e+00,4.385548e+14,0.695238,0.332718,0.724836,35.454005,56.174894,"Hidden Size=[4], regularizer=0.02, learning_ra..."
2,model_1_3_7,0.817910,0.087649,-1.181859,0.685117,0.452227,1.828980,1.608931,1.282700e+00,4.314823e+14,0.672478,0.375691,0.701107,35.587141,56.308030,"Hidden Size=[4], regularizer=0.02, learning_ra..."
3,model_1_4_2,0.827391,0.081482,-1.198831,0.870456,0.428680,1.841343,1.621446,6.277446e-02,4.228255e+14,0.654737,0.408198,0.682610,35.694087,56.414976,"Hidden Size=[4], regularizer=0.02, learning_ra..."
4,model_1_4_6,0.839168,0.072785,-1.222784,0.888564,0.399432,1.858778,1.639110,5.399978e-02,4.116460e+14,0.632006,0.448577,0.658912,35.835425,56.556314,"Hidden Size=[4], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,model_4_29_9,0.847330,-0.609202,-3.295348,1.000000,0.379160,3.225951,3.167445,2.214995e-10,5.402395e+09,0.615760,1.050193,0.641974,195.939593,314.170548,"Hidden Size=[24], regularizer=0.02, learning_r..."
701,model_4_29_6,0.847330,-0.609202,-3.295348,1.000000,0.379160,3.225951,3.167445,2.214995e-10,5.402395e+09,0.615760,1.050193,0.641974,195.939593,314.170548,"Hidden Size=[24], regularizer=0.02, learning_r..."
702,model_4_3_2,0.847330,-0.609202,-3.295348,1.000000,0.379160,3.225951,3.167445,2.678409e-10,5.402395e+09,0.615760,1.050193,0.641974,195.939593,314.170548,"Hidden Size=[24], regularizer=0.02, learning_r..."
703,model_4_3_9,0.847330,-0.609202,-3.295348,1.000000,0.379160,3.225951,3.167445,2.678409e-10,5.402395e+09,0.615760,1.050193,0.641974,195.939593,314.170548,"Hidden Size=[24], regularizer=0.02, learning_r..."
